In [2]:
import soccerdata as sd
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm

[03/18/24 17:53:47] INFO     No custom team name replacements found. You can configure these in       ]8;id=179095;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=373907;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\gfuen\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=507059;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=710767;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#283\283]8;;\
                             C:\Users\gfuen\soccerdata\config\league_dict.json.                                    

In [3]:
df = pd.read_csv("datos_fotmob_completo.csv")

In [5]:
df.columns

Index(['id', 'date', 'leagueName', 'homeTeam', 'awayTeam', 'homeTeam_score',
       'awayTeam_score', 'homeIdTeam', 'awayIdTeam', 'parentLeagueId',
       'leagueId', 'BallPossesion_home', 'BallPossesion_away',
       'ShotsOnTarget_home', 'ShotsOnTarget_away', 'ShotsOffTarget_home',
       'ShotsOffTarget_away', 'fouls_home', 'fouls_away', 'Offsides_home',
       'Offsides_away', 'corners_home', 'corners_away', 'total_shots_home',
       'total_shots_away', 'big_chance_home', 'big_chance_away',
       'big_chance_missed_title_home', 'big_chance_missed_title_away',
       'accurate_passes_home', 'accurate_passes_away', 'shots_home',
       'shots_away', 'blocked_shots_home', 'blocked_shots_away',
       'shots_woodwork_home', 'shots_woodwork_away', 'shots_inside_box_home',
       'shots_inside_box_away', 'shots_outside_box_home',
       'shots_outside_box_away', 'passes_home', 'passes_away',
       'own_half_passes_home', 'own_half_passes_away',
       'opposition_half_passes_home', 'o